$\DeclareMathOperator*{\E}{\mathbb{E}}$
$\DeclareMathOperator*{\Q}{\mathbb{Q}}$


$\DeclareMathOperator*{\E}{\mathbb{E}} % expectation
\DeclareMathOperator*{\Q}{\mathbb{Q}} % Q probability measure
\DeclareMathOperator*{\F}{{\mathcal{F}}}
\DeclareMathOperator{\d}{\mathrm{d}\!}  % differential
$

<center>
  
# Pricing Options Using Stochastic Volatility Models 
 
## Monte Carlo Simulations in NumPy, TensorFlow and PyTorch 
  
 ### Ferenc Szalma
  
 ### New York, NY

 ### May 3, 2019



This hands-on notebook discusses how to price equity options via integrating stochastic differential equations (SDE) using Monte Carlo (MC) simulations. We choose the Heston model, a mean-reversion stochastic volatility model, as an example to show how the machinery of SDEs, their intergration over time and Monte Carlo simulations go together to price equity options. We include several versions of the Python code which show the setup using, in the simplest case, for-loops, then vectorized versions in NumPy, and also using modern tensor libraries, such as TensorFlow and PyTorch for the sake of coding and speed comparisons.

# 1 - Introduction
**Stochastic calculus** grew out of the field of the physical phenomena of diffusion [Boltzmann 1872, Einstein 1905] and was first applied to finance by Merton [Merton 1969, 1973] to understand how prices are set in the market under the classical assumption of 'equilibrium'. The celebrated Black-Scholes (BS) options pricing formula [Black and Scholes 1973], merited by a Nobel prize in 1997 in Economics, dates back to diffusion applied to contingent claims, i.e., options, when the diffusion is a Markov process. By changing the diffusion (or stochastic) paths and the underlying probability measure of the stochastic paths (which we call the pricing measure $\Q$) according to Girsanov's theorem [Girsanov 1960], the price of an option turns out to be a simple mathematical expactation value of the pay-off of the option at the expiration of that option. This is called martingale pricing since the option's price process is a martingale stochastic process, which keeps its expectation constant over time (again, this is all true only in the pricing measure), which, in turn, equals to its initial value. This diffusion or stochastic process is characterized by a single parameter, its volatility, $\sigma$, and, surprisingly, it does not depend on the (mean) expected return of the underlying stock contrary to what one could have thought before Black and Scholes' work. BS assumed that the volatility $\sigma$ is constant in time, however, this is not quite true in real life, so the BS theory needs modification. Market participants have noticed a so called volatility smile, which is changing volatility across various stikes of the option having a shape of a "smile" or "smirk." This volatility smile can be modelled if we assume that the volatility chages in time, namely it is a stochastic process itself. So the stock price process is not driven by a single stochastic differential equation but by two, one for the price and another one for the volatility that is coupled with the first, price process.

Before going further and discussing the other ingredient of pricing options with MC simulations, note that, as the above paragraph shows, options pricing is not a run of the mill high school math problem, you need to patiently and carefully follow the math and the code below. Now let's move on to the other two ingredients: Monte Carlo simulations and Euler integration of differential equations.

**Monte Carlo simulations** have been around as long as the atomic bomb, they are crucial in integrating functions in high dimensions by stochastically walking around in that space and making sums or averages. Indeed, MC simulation were developed in the 1940's by [Metropolis, Teller, Ulam, von Neumann] to simulate the flow (kind of diffusion) of material in an A-bomb as it reaches criticality (or to make sure it reaches criticality) and becomes less of an atomic reactor with controlled heat production but a runaway fission process producing the most energy possible in a very short amount of time. We want to use MC simulations for much tamer purposes here. Our stochastic processes are driven by Brownian motion, a continuous random process whose distribution is normal at any time, so at each time step the stock price moves such that the size of its movement comes from a normal distribution, or more precisely a function of the normal distribution where the function is determined by the specific integration technique being used (see next paragraph). These consecutive independent random steps constitute a Markov chain of the Monte Carlo process. 

There are many finite difference methods to integrate SDEs, less precise first order methods and more sophisticated higher order methods, which are more precise but also more convoluted. We will integrate the SDEs using the Euler method, which is a first order integration method. Euler approximately solved ordinary differential equations the simplest ways with his method in the 1760's and Maruyama [Maruyama 196.] adapted Euler's method to SDEs two centuries later. The SDE is driven by Brownian motion in the **Euler-Maruyama integration scheme**, at each time step the stock price moves by a random number pulled from an IID normal distribution as mentioned in the previous paragraph. Higher order methods such as the second order Milstein [Milstein 1976] method or Alfonsi's implicit method [Alfonzi 2010] are more difficult to picture as the normal random variable is embedded in a scheme that advances the SDE but may be more efficient when speed is really important.


In the following:
* Heston model
* Monte Carlo simulation
* Euler scheme
* for-loop based code
* vectorized in NumPy
* vectorized in TensorFlow
* vectorized in PyTorch
* On the quantum computer
* Reinforcement learning based solution







# Heston-stochastic-volatility-model

In the notebooks I'll discuss various implementations of the Heston stochastic volatility model of pricing European call options. 

The Heston model goes beyond the Black-Scholes options pricing model in the sense that the constant volatility of the Black-Scholes model is replaced by a stochastic process for the volatility. This simple trick makes it possible to account for volatility smiles often seen in practice.

## Mathematical description of the model

The Heston stochastic volatility model is described by two coupled stochastic differential equations: one for the stock process, and another for the volatility process both driven by their respective Brownian motions:

\begin{align} 
\d S_t & = \sqrt{v_t} S_t \d B_t, \qquad S_0 > 0 \\
\d v_t & = \kappa (\theta-r_t) \d t + \sigma\sqrt{v_t}(\rho \d B_t + \sqrt{1-\rho^2} \d W_t), \qquad v_0 > 0.
\end{align}

In these two coupled stochastic differential equations:
* $S_t$ is stock price process driven by the $B_t$ Brownian motion, 
* $v_t$ is the volatility process driven by $W_t$ Brownian motion ~~mixed with $B_t$ so that the volatility is driven by a Brownian motion that has $\rho$ correlation with $B_t$. $W_t$ is independent of $B_t$~~.

The stock price process is not a geometric Brownian motion since the volatility is a non-adapted stochastic process (instead of being constant or deterministic as in the Black-Scholes case).

The volatility process is a CIR process, a mean reversion process which has
* mean-reversion level $\theta$
* the volatility mean revert to this level at the mean-reversion speed $\kappa$
* $\sigma$ is volatility of the volatility process (vol of vol).

$B$ and $W$ are two independent $\Q$-Brownian motions generating the filtration $\F_t$ in the ($\Omega$, $\F_t$, $\Q$) probability space, where $\Q$ is the pricing measure.

Time $t$ is in the $[0, T]$ interval where $T$ is the expiry time of the option measured in years.

The price of a call option, whose strike price is $K$, $c(t, s, v)$ is the conditional expectation of the final pay-off (random variable) $(S_T - K)^+$  given the $\F_t$ filtration at time $t$, i.e., given all information up to $t$ in the $\Q$ pricing measure:

$$ c(t, S_t, v_t) = {\E}^{\Q}[(S_T - K)^+|{\F}_t] $$

In the following Monte Carlo simulation, I'll generate the path of the stock price $S_t$ and its volatility, $v_t$ from $t=0$ to $T$, so $\F_t = \F_0$, that is, no information is known after $t=0$, hence the conditional expectation above becomes a simple expectation with no condition, since ${\E}^{\Q}[(S_T - K)^+|{\F}_{t=0}] = {\E}^{\Q}[(S_T - K)^+]$




## Monte Carlo simulation

I'll show how to integrate the coupled SDE's using the simplest integration method, the Euler method, generating MC paths and then making averages.

In the first pass, I'll show the MC path generation using for loops to make things more accessible, and later give a vectorized solution which can utilize the capabilities of CPU's or GPU's vector units making the algorithm 100x (?) faster.

So, let's discretize the the SDEs: 
* $\Delta$ is the time step 
* $Z'$   standard normal distributions for the $S_t$ process and 
* $Z''$ standard normal distribution independent of $Z'$, which helps make
* $Z$ driving $v_t$ where $Z = \rho Z' + \sqrt{1-\rho^2}Z''$ correlated standard normal -- its correlation with $Z'$ is $\rho$

Since $v_t$ cannot be negative, I'll replace a negative $v$ with $v=0$, which is called the absorbing way. (It is left to the reader as an exercise to try the reflection way, when a negative $v$ is replaced by $v=-v$. Try and see what you get using this way.)

### Explicit Euler scheme

\begin{align}
S_{i+1} & = S_i + \sqrt{v_i} S_i \sqrt{\Delta} Z'   = S_i \exp[\sqrt{v_i}\sqrt{\Delta} Z' - v_i \Delta /2]\\
v_{i+1} & = v_i + \kappa (\theta - v_i) \Delta + \sigma\sqrt{v_i}\sqrt{\Delta}Z
\end{align}


#### For-loop based Monte Carlo

In [1]:
import random
from math import sqrt, exp

In [2]:
random.seed(42)  # Seed random number generator

In [3]:
N = 1000    # Number of time steps
n = 10000  # Number of MC paths (10,000 20sec on MBP '18 1 thread)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [4]:
# Integrate equations:
#   Euler method
#   for loop based (not vectorized)
Stv = [0.] * n  # Reserve St vector of size n
for i in range(n):  # i-th MC path
  vt = v0
  St = S0
  for t in range(1,N):  # Generate a MC path

    Zs = random.gauss(0.,1.)  # Generate rnd numbers (Mersenne Twister algo) for S_t process
    Zv = rho * Zs + sqrt(1 - rho**2.) * random.gauss(0., 1.)  # Rnd number for v_t process

    # Euler integration
    vt = max(vt, 0.)
    St = St * exp( sqrt(vt * dt) * Zs - vt * dt / 2.)          # Stock price process
    vt = vt + k * (th - vt) * dt + sig * sqrt(vt * dt) * Zv    # Volatility process
  Stv[i] = St  # Store S_T's

In [5]:
sum_m = lambda x: max(x - K, 0.)
sum_s = lambda x: max(x - K, 0.) ** 2.

mean_price = sum(map(sum_m, Stv)) / n
std_error_price  = sqrt(sum(map(sum_s, Stv)) / (n * (n - 1.)))

In [6]:
mean_price, std_error_price

(3.91400889505858, 0.06870932165948272)

#### Vectorized Monte Carlo / Numpy

In [7]:
# Quick numpy/python refresher from Karpathy at Stanford 
#   http://cs231n.github.io/python-numpy-tutorial/
# Quick comparison of numpy and Matlab, which might be useful 
# if you are coming from the Matlab world 
#   https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html
import numpy as np

In [8]:
np.random.seed(42)  # Seed random number generator

In [9]:
N = 1000    # Number of time steps
n = 1000000  # Number of MC paths (1,000,000 2min on Colab; 80sec on MBP18 6 threads)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [10]:
# Integrate equations:
#   Euler method
#   MC vectorized
vt = np.ones(n) * v0
St = np.ones(n) * S0
for t in range(1,N):  # Generate MC paths

  Zs = np.random.normal(size=n)  # Generate rnd numbers (??? rnd gen) for S_t process
  Zv = rho * Zs + np.sqrt(1 - rho**2.) * np.random.normal(size=n)  # Rnd numbers for v_t process

  # Euler integration
  vt = np.maximum(vt, 0.)
  St = St * np.exp( np.sqrt(vt * dt) * Zs - vt * dt / 2.)     # Stock price process
  vt = vt + k * (th - vt) * dt + sig * np.sqrt(vt * dt) * Zv  # Volatility process

mean_price = np.mean(np.maximum(St - K, 0.))
std_error_price  = np.std(np.maximum(St - K, 0.)) / np.sqrt(n)

In [11]:
mean_price, std_error_price

(3.9139916791666054, 0.005651208214228756)

#### Vectorized Monte Carlo / TensorFlow

In [12]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
tf.version.VERSION

'1.13.1'

In [13]:
tf.executing_eagerly()

True

In [14]:
tf.random.set_random_seed(42)

In [15]:
N = 1000    # Number of time steps
n = 1000000  # Number of MC paths (1,000,000  30sec on Colab, 22sec on MBP '18 6 threads)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [16]:
# Integrate equations:
#   Euler method
#   MC vectorized
vt = tf.ones(n) * v0
St = tf.ones(n) * S0
for t in range(1,N):  # Generate MC paths

  Zs = tf.random.normal([n])  # Generate rnd numbers (??? rnd gen) for S_t process
  Zv = rho * Zs + tf.sqrt(1 - rho**2.) * tf.random.normal([n])  # Rnd numbers for v_t process

  # Euler integration
  vt = tf.maximum(vt, 0.)
  St = St * tf.exp( tf.sqrt(vt * dt) * Zs - vt * dt / 2.)     # Stock price process
  vt = vt + k * (th - vt) * dt + sig * tf.sqrt(vt * dt) * Zv  # Volatility process

mean_price = tf.reduce_sum(tf.maximum(St - K, 0.)) / n
std_error_price  = np.std(tf.maximum(St - K, 0.)) / tf.sqrt(float(n))

In [17]:
mean_price, std_error_price

(<tf.Tensor: id=44972, shape=(), dtype=float32, numpy=3.9317935>,
 <tf.Tensor: id=44981, shape=(), dtype=float32, numpy=0.0056690983>)

#### Vectorized Monte Carlo / PyTorch

In [18]:
import torch
import numpy as np

In [19]:
torch.__version__

'1.0.1.post2'

In [20]:
torch.manual_seed(42)

In [21]:
N = 1000    # Number of time steps
n = 1000000  # Number of MC paths (1,000,000  37sec on Colab, 24sec on MBP '18 1 thread)

S0 = 100.   # Initial price of share of stock
K = 100.    # Strike price of option
v0 = 0.01   # Initial volatility
th = 0.01   # Long time average of volatility
k = 2.      # kappa mean reversion speed
sig = 0.1   # vol of vol

rho = -.5   # correlation coefficient of Z' and Z
T = 1.      # expiration time from today

dt = T/N    # Time step

In [22]:
# Integrate equations:
#   Euler method
#   MC vectorized
vt = torch.ones(n) * v0
St = torch.ones(n) * S0
for t in range(1,N):  # Generate MC paths

  Zs = torch.randn(n)  # Generate rnd numbers (??? rnd gen) for S_t process
  Zv = rho * Zs + np.sqrt(1 - rho**2.) * torch.randn(n)  # Rnd numbers for v_t process

  # Euler integration
  vt = torch.max(vt, torch.zeros(n))
  St = St * torch.exp( torch.sqrt(vt * dt) * Zs - vt * dt / 2.)     # Stock price process
  vt = vt + k * (th - vt) * dt + sig * torch.sqrt(vt * dt) * Zv  # Volatility process

mean_price = torch.mean(torch.max(St - K, torch.zeros(n)))
std_error_price  = torch.std(torch.max(St - K, torch.zeros(n))) / np.sqrt(float(n))

In [23]:
mean_price, std_error_price

(tensor(3.9181), tensor(0.0057))